In [2]:
!/opt/venv/bin/python -m pip install --upgrade pip
# Handling pip upgrades

import pandas as pd
import numpy as np
import math

# Making plotly as the backend for pandas
!pip install plotly
pd.options.plotting.backend = "plotly"

# Setting the theme
import plotly.io as pio
import plotly.express as px
pio.templates.default = "plotly_white"

from sklearn.model_selection import train_test_split
import pprint

     |████████████████████████████████| 1.5 MB 3.1 MB/s eta 0:00:01     |███████████████████████████████▏| 1.5 MB 3.1 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2.3
    Uninstalling pip-20.2.3:
      Successfully uninstalled pip-20.2.3
     |████████████████████████████████| 13.1 MB 3.2 MB/s eta 0:00:01    |██████████                      | 4.0 MB 3.2 MB/s eta 0:00:03
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=a10ee4c419781a32860448a447a2bc90f00aba0ffbc5e041e1a5b8e5c1405e19
  Stored in directory: /home/jovyan/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [32]:
iris_set = pd.read_csv("./iris/iris.data")
# add the column names, as it doesn't have it
attributes = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]
iris_set.columns = attributes
iris_set

,sepal_length,sepal_width,petal_length,petal_width,species
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,Iris-virginica
145,6.3,2.5,5.0,1.9,Iris-virginica
146,6.5,3.0,5.2,2.0,Iris-virginica
147,6.2,3.4,5.4,2.3,Iris-virginica


In [33]:
iris_set.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [34]:
labels= iris_set['species'].unique()
print(labels)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [35]:
# iris_set.species = iris_set.species.apply(lambda x: int(np.where(labels == x)[0]))

In [36]:
iris_set

,sepal_length,sepal_width,petal_length,petal_width,species
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,Iris-virginica
145,6.3,2.5,5.0,1.9,Iris-virginica
146,6.5,3.0,5.2,2.0,Iris-virginica
147,6.2,3.4,5.4,2.3,Iris-virginica


In [71]:
#split training and testing dataset
train, test = train_test_split(iris_set,test_size=0.2)

In [74]:
print(len(train))
print(len(test))
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

119
30


In [75]:
train

,sepal_length,sepal_width,petal_length,petal_width,species
0,4.8,3.4,1.9,0.2,Iris-setosa
1,6.3,2.8,5.1,1.5,Iris-virginica
2,6.1,2.6,5.6,1.4,Iris-virginica
3,4.9,2.5,4.5,1.7,Iris-virginica
4,5.8,2.7,4.1,1.0,Iris-versicolor
...,...,...,...,...,...
114,5.9,3.0,4.2,1.5,Iris-versicolor
115,6.7,2.5,5.8,1.8,Iris-virginica
116,6.5,3.2,5.1,2.0,Iris-virginica
117,5.6,3.0,4.5,1.5,Iris-versicolor


In [76]:
class DecisionTree(object):
    def __init__(self, _max_depth, _min_splits):
        self.max_depth = _max_depth
        self.min_splits = _min_splits

    def fit(self, _feature, _label):
        
        self.feature = _feature
        self.label = _label
        self.train_data = np.column_stack((self.feature,self.label))
        self.build_tree()


    def compute_gini_similarity(self, groups, class_labels):
        """
        compute the gini index for the groups and class labels

        :param groups:
        :param class_labels:
        :return:
        """
        num_sample = sum([len(group) for group in groups])
        gini_score = 0

        for group in groups:
            size = float(len(group))

            if size == 0:
                continue
            score = 0.0
            for label in class_labels:
                porportion = (group[:,-1] == label).sum() / size
                score += porportion * porportion
            gini_score += (1.0 - score) * (size/num_sample)

        return gini_score

    def terminal_node(self, _group):
        """
        Function set terminal node as the most common class in the group to make prediction later on
        is an helper function used to mark the leaf node in the tree based on the early stop condition
        or actual stop condition which ever is meet early
        :param _group:
        :return:
        """
        class_labels, count = np.unique(_group[:,-1], return_counts= True)
        return class_labels[np.argmax(count)]

    def split(self, index, val, data):
        """
        split features into two groups based on their values
        :param index:
        :param val:
        :param data:
        :return:
        """
        data_left = np.array([]).reshape(0,self.train_data.shape[1])
        data_right = np.array([]).reshape(0, self.train_data.shape[1])

        for row in data:
            if row[index] <= val :
                data_left = np.vstack((data_left,row))

            if row[index] > val:
                data_right = np.vstack((data_right, row))

        return data_left, data_right

    def best_split(self, data):
        """
        find the best split information using the gini score
        :param data:
        :return best_split result dict:
        """
        class_labels = np.unique(data[:,-1])
        best_index = 999
        best_val = 999
        best_score = 999
        best_groups = None

        for idx in range(data.shape[1]-1):
            for row in data:
                groups = self.split(idx, row[idx], data)
                gini_score = self.compute_gini_similarity(groups,class_labels)

                if gini_score < best_score:
                    best_index = idx
                    best_val = row[idx]
                    best_score = gini_score
                    best_groups = groups
        result = {}
        result['index'] = best_index
        result['val'] = best_val
        result['groups'] = best_groups
        return result


    def split_branch(self, node, depth):
        """
        recursively split the data and
        check for early stop argument based on self.max_depth and self.min_splits
        - check if left or right groups are empty is yess craete terminal node
        - check if we have reached max_depth early stop condition if yes create terminal node
        - Consider left node, check if the group is too small using min_split condition
            - if yes create terminal node
            - else continue to build the tree
        - same is done to the right side as well.
        else
        :param node:
        :param depth:
        :return:
        """
        left_node , right_node = node['groups']
        del(node['groups'])

        if not isinstance(left_node,np.ndarray) or not isinstance(right_node,np.ndarray):
            node['left'] = self.terminal_node(left_node + right_node)
            node['right'] = self.terminal_node(left_node + right_node)
            return

        if depth >= self.max_depth:
            node['left'] = self.terminal_node(left_node)
            node['right'] = self.terminal_node(right_node)
            return

        if len(left_node) <= self.min_splits:
            node['left'] = self.terminal_node(left_node)
        else:
            node['left'] = self.best_split(left_node)
            self.split_branch(node['left'],depth + 1)


        if len(right_node) <= self.min_splits:
            node['right'] = self.terminal_node(right_node)
        else:
            node['right'] = self.best_split(right_node)
            self.split_branch(node['right'],depth + 1)

    def build_tree(self):
        """
        build tree recursively with help of split_branch function
         - Create a root node
         - call recursive split_branch to build the complete tree
        :return:
        """
        self.root = self.best_split(self.train_data)
        self.split_branch(self.root, 1)
        return self.root

    def _predict(self, node, row):
        """
        Recursively traverse through the tress to determine the
        class of unseen sample data point during prediction
        :param node:
        :param row:
        :return:
        """
        if row[node['index']] < node['val']:
            if isinstance(node['left'], dict):
                return self._predict(node['left'], row)
            else:
                return node['left']

        else:
            if isinstance(node['right'],dict):
                return self._predict(node['right'],row)
            else:
                return node['right']

    def predict(self, test_data):
        """
        predict the set of data point
        :param test_data:
        :return:
        """
        self.predicted_label = np.array([])
        for idx in test_data:
            self.predicted_label = np.append(self.predicted_label, self._predict(self.root,idx))

        return self.predicted_label

In [77]:
decision_tree_model = DecisionTree(_max_depth = 2, _min_splits = 30)
decision_tree_model.fit(train.drop('species', axis=1), train['species'])
prediction  = decision_tree_model.predict(test.drop('species', axis=1).to_numpy())

In [84]:
pd.Series(prediction)

0     Iris-versicolor
1      Iris-virginica
2      Iris-virginica
3      Iris-virginica
4      Iris-virginica
5      Iris-virginica
6         Iris-setosa
7         Iris-setosa
8     Iris-versicolor
9      Iris-virginica
10     Iris-virginica
11        Iris-setosa
12        Iris-setosa
13    Iris-versicolor
14     Iris-virginica
15        Iris-setosa
16    Iris-versicolor
17     Iris-virginica
18    Iris-versicolor
19     Iris-virginica
20     Iris-virginica
21    Iris-versicolor
22        Iris-setosa
23    Iris-versicolor
24        Iris-setosa
25        Iris-setosa
26    Iris-versicolor
27        Iris-setosa
28        Iris-setosa
29    Iris-versicolor
dtype: object

In [81]:
test.predicted = pd.Series(prediction)

/opt/venv/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [82]:
test

,sepal_length,sepal_width,petal_length,petal_width,species,predicted
0,6.2,2.9,4.3,1.3,Iris-versicolor,Iris-versicolor
1,7.2,3.2,6.0,1.8,Iris-virginica,Iris-virginica
2,6.7,3.3,5.7,2.5,Iris-virginica,Iris-virginica
3,6.3,2.5,4.9,1.5,Iris-versicolor,Iris-virginica
4,6.9,3.2,5.7,2.3,Iris-virginica,Iris-virginica
5,5.6,2.8,4.9,2.0,Iris-virginica,Iris-virginica
6,4.9,3.1,1.5,0.1,Iris-setosa,Iris-setosa
7,4.6,3.1,1.5,0.2,Iris-setosa,Iris-setosa
8,6.7,3.1,4.4,1.4,Iris-versicolor,Iris-versicolor
9,6.0,3.0,4.8,1.8,Iris-virginica,Iris-virginica


In [83]:
test['species'].isin(test['predicted']).value_counts()

True    30
Name: species, dtype: int64

In [85]:
# 100% accuracy, so Precision, Recall and sensitivity is 1